In [96]:
import sys
import numpy as np
import surprise
from surprise import AlgoBase, Dataset, evaluate, Reader, accuracy
from surprise.prediction_algorithms.knns import KNNBasic
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
data = Dataset.load_builtin('ml-1m')

Dataset ml-1m could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-1m.zip...
Done! Dataset ml-1m has been saved to /Users/alexworth/.surprise_data/ml-1m


In [97]:
class GlobalMean(AlgoBase):
    def train(self, trainset):

        # Here again: call base method before doing anything.
        AlgoBase.train(self, trainset)

        # Compute the average rating
        self.the_mean = np.mean([r for (_, _, r) in self.trainset.all_ratings()])

    def estimate(self, u, i):
        #print(u,i)
        return self.the_mean
    
    def topandas(self,u,i):
        pass

In [98]:
class MeanofMeans(AlgoBase):
    def train(self, trainset):

        # Here again: call base method before doing anything.
        AlgoBase.train(self, trainset)

        users = np.array([u for (u, _, _) in self.trainset.all_ratings()])
        items = np.array([i for (_, i, _) in self.trainset.all_ratings()])
        ratings = np.array([r for (_, _, r) in self.trainset.all_ratings()])

        user_means,item_means = {},{}
        for user in np.unique(users):
            user_means[user] = ratings[users==user].mean()
        for item in np.unique(items):
            item_means[item] = ratings[items==item].mean()

        self.global_mean = ratings.mean()    
        self.user_means = user_means
        self.item_means = item_means
        
    def estimate(self, u, i):
        """
        return the mean of means estimate
        """
        
        if u not in self.user_means:
            return(np.mean([self.global_mean,
                            self.item_means[i]]))

        if i not in self.item_means:
            return(np.mean([self.global_mean,
                            self.user_means[u]]))

        return(np.mean([self.global_mean,
                        self.user_means[u],
                        self.item_means[i]]))
    
    def prediction(self, u, i):
        
        return algo.predict(u, i, r_ui=4, verbose=True)

In [99]:
print("\nGlobal Mean...")
algo = GlobalMean()
evaluate(algo, data)
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

print("\nMeanOfMeans...")
algo = MeanofMeans()
evaluate(algo, data)
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

# print("\nKNNBasic...")
# algo = KNNBasic()
# evaluate(algo,data)


Global Mean...
Evaluating RMSE, MAE of algorithm GlobalMean.

------------
Fold 1
RMSE: 1.1160
MAE:  0.9333
------------
Fold 2
RMSE: 1.1176
MAE:  0.9347
------------
Fold 3
RMSE: 1.1177
MAE:  0.9338
------------
Fold 4
RMSE: 1.1184
MAE:  0.9346
------------
Fold 5
RMSE: 1.1158
MAE:  0.9329
------------
------------
Mean RMSE: 1.1171
Mean MAE : 0.9339
------------
------------
user: 9          item: 1          r_ui = 4.00   est = 3.58   {'was_impossible': False}

MeanOfMeans...
Evaluating RMSE, MAE of algorithm MeanofMeans.

------------
Fold 1
RMSE: 0.9960
MAE:  0.8145
------------
Fold 2
RMSE: 0.9971
MAE:  0.8167
------------
Fold 3
RMSE: 0.9971
MAE:  0.8147
------------
Fold 4
RMSE: 0.9985
MAE:  0.8164
------------
Fold 5
RMSE: 0.9966
MAE:  0.8148
------------
------------
Mean RMSE: 0.9970
Mean MAE : 0.8154
------------
------------
user: 9          item: 1          r_ui = 4.00   est = 3.83   {'was_impossible': False}


In [12]:
type(data)

surprise.dataset.DatasetAutoFolds

In [31]:
perf = evaluate(algo, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm MeanofMeans.

------------
Fold 1
RMSE: 1.0183
MAE:  0.8391
------------
Fold 2
RMSE: 1.0123
MAE:  0.8322
------------
Fold 3
RMSE: 1.0195
MAE:  0.8390
------------
Fold 4
RMSE: 1.0234
MAE:  0.8437
------------
Fold 5
RMSE: 1.0143
MAE:  0.8344
------------
------------
Mean RMSE: 1.0176
Mean MAE : 0.8377
------------
------------


In [32]:
uid = str(200)
iid = str(300)
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

user: 200        item: 300        r_ui = 4.00   est = 3.75   {'was_impossible': False}


In [33]:
for x in range(10):
    uid = str(x)
    iid = str(1)
    pred = algo.predict(uid, iid, r_ui=4, verbose=True)

user: 0          item: 1          r_ui = 4.00   est = 3.70   {'was_impossible': False}
user: 1          item: 1          r_ui = 4.00   est = 3.66   {'was_impossible': False}
user: 2          item: 1          r_ui = 4.00   est = 3.70   {'was_impossible': False}
user: 3          item: 1          r_ui = 4.00   est = 3.43   {'was_impossible': False}
user: 4          item: 1          r_ui = 4.00   est = 3.98   {'was_impossible': False}
user: 5          item: 1          r_ui = 4.00   est = 3.43   {'was_impossible': False}
user: 6          item: 1          r_ui = 4.00   est = 3.66   {'was_impossible': False}
user: 7          item: 1          r_ui = 4.00   est = 3.79   {'was_impossible': False}
user: 8          item: 1          r_ui = 4.00   est = 3.72   {'was_impossible': False}
user: 9          item: 1          r_ui = 4.00   est = 3.92   {'was_impossible': False}


In [47]:
type(pred)

surprise.prediction_algorithms.predictions.Prediction

In [100]:
#data = Dataset.load_builtin('ml-100k')
data.split(n_folds=5)

algo = MeanofMeans()

for trainset, testset in data.folds():

    # train and test algorithm.
    algo.train(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    rmse = accuracy.rmse(predictions, verbose=True)

RMSE: 0.9980
RMSE: 0.9985
RMSE: 0.9960
RMSE: 0.9964
RMSE: 0.9964


In [51]:
type(predictions)

list

In [52]:
predictions[:10]

[Prediction(uid='642', iid='1287', r_ui=2.0, est=2.7219404557182822, details={'was_impossible': False}),
 Prediction(uid='222', iid='225', r_ui=1.0, est=3.1903020190436124, details={'was_impossible': False}),
 Prediction(uid='109', iid='527', r_ui=3.0, est=3.6956132516413684, details={'was_impossible': False}),
 Prediction(uid='346', iid='38', r_ui=3.0, est=3.321943245507653, details={'was_impossible': False}),
 Prediction(uid='435', iid='756', r_ui=3.0, est=3.2625399297030104, details={'was_impossible': False}),
 Prediction(uid='894', iid='313', r_ui=4.0, est=3.7940573451239321, details={'was_impossible': False}),
 Prediction(uid='379', iid='133', r_ui=4.0, est=3.8011155406541639, details={'was_impossible': False}),
 Prediction(uid='588', iid='107', r_ui=5.0, est=3.5717457019356029, details={'was_impossible': False}),
 Prediction(uid='848', iid='25', r_ui=5.0, est=3.785934887726206, details={'was_impossible': False}),
 Prediction(uid='883', iid='69', r_ui=2.0, est=3.7861857223030513, 

In [53]:
len(predictions)

33333

In [54]:
type(trainset)

surprise.dataset.Trainset

In [55]:
predictions[:1]

[Prediction(uid='642', iid='1287', r_ui=2.0, est=2.7219404557182822, details={'was_impossible': False})]

In [101]:
for x in predictions[0]:
    print(x)

3017
550
3.0
3.30825097883
{'was_impossible': False}


In [104]:
data = [np.array(x) for x in predictions]

In [106]:
print (a[2],a[3])

['109' '527' 3.0 3.6956132516413684 {'was_impossible': False}] ['346' '38' 3.0 3.321943245507653 {'was_impossible': False}]


In [63]:
a[0]

array(['642', '1287', 2.0, 2.7219404557182822, {'was_impossible': False}], dtype=object)

In [69]:
names = np.array(range(5))

In [ ]:
pd.DataFrame(data=data[1:,1:],    # values
...              index=data[1:,0],    # 1st column as index
...              columns=data[0,1:])

In [105]:
data2 = np.array(data)

In [82]:
data2

array([['642', '1287', 2.0, 2.7219404557182822, {'was_impossible': False}],
       ['222', '225', 1.0, 3.1903020190436124, {'was_impossible': False}],
       ['109', '527', 3.0, 3.6956132516413684, {'was_impossible': False}],
       ['346', '38', 3.0, 3.321943245507653, {'was_impossible': False}],
       ['435', '756', 3.0, 3.2625399297030104, {'was_impossible': False}],
       ['894', '313', 4.0, 3.7940573451239321, {'was_impossible': False}],
       ['379', '133', 4.0, 3.8011155406541639, {'was_impossible': False}],
       ['588', '107', 5.0, 3.5717457019356029, {'was_impossible': False}],
       ['848', '25', 5.0, 3.785934887726206, {'was_impossible': False}],
       ['883', '69', 2.0, 3.7861857223030513, {'was_impossible': False}],
       ['860', '900', 3.0, 3.4983953400794938, {'was_impossible': False}],
       ['433', '276', 5.0, 3.5786366883106164, {'was_impossible': False}],
       ['396', '291', 4.0, 3.4856631165377814, {'was_impossible': False}],
       ['815', '175', 3.0, 3.

In [107]:
df = pd.DataFrame(data2,columns=['userid','itemid','rating','prediction','was_impossible'])

In [108]:
df= df[['userid','itemid','rating','prediction']]

In [109]:
df.head(5)

,userid,itemid,rating,prediction
0,3017,550,3,3.30825
1,3727,2405,2,3.31767
2,5493,380,3,3.50634
3,1051,673,4,3.22945
4,4524,1840,1,3.41057


In [111]:
df.to_csv('preds1m.csv')

In [110]:
len(df)

200041